# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Import for adding the marker layer on second map
import json 
import pprint
 

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_data_file = "../Resources/Cities.csv"
cities_data = pd.read_csv(cities_data_file)

cities_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Mataura,-46.1927,168.8643,73.99,44,28,17.00,NZ,1615695560
1,Ilulissat,69.2167,-51.1000,8.60,56,100,13.80,GL,1615695763
2,Albany,42.6001,-73.9662,33.01,50,1,5.14,US,1615695800
3,Aklavik,68.2191,-135.0107,-13.00,77,24,6.91,CA,1615695800
4,Punta Arenas,-53.1500,-70.9167,44.01,65,0,3.44,CL,1615695801


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

city_coordinates = cities_data[["Lat", "Lng"]]
cities_humidity = cities_data["Humidity"]

fig = gmaps.figure(center=(50.0, -35.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(city_coordinates, weights=cities_humidity,
                                  dissipating=False, max_intensity=10, 
                                  point_radius=2)

fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
perfect_temperature = cities_data.loc[(cities_data["Max Temp"] > 67) &
                                      (cities_data["Max Temp"] < 78) &
                                      (cities_data["Wind Speed"] < 10) &
                                      (cities_data["Cloudiness"] > 10) &
                                      (cities_data["Cloudiness"] <= 20), :]
                                     
perfect_temperature = perfect_temperature.dropna(how='any')  
perfect_temperature

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
118,Touros,-5.1989,-35.4608,75.20,83,20,3.44,BR,1615695843
150,Sokoni,-6.4333,39.5500,77.00,88,20,0.78,TZ,1615695855
171,East London,-33.0153,27.9116,71.60,78,20,5.75,ZA,1615695872
238,Maxixe,-23.8597,35.3472,72.88,84,17,6.20,MZ,1615695896
294,Pisco,-13.7000,-76.2167,72.00,88,20,3.44,PE,1615695928
459,Vila Velha,-20.3297,-40.2925,73.99,88,20,2.30,BR,1615695686
469,El Salvador,13.8333,-88.9167,77.00,83,20,1.14,SV,1615695982
493,Tahoua,14.8888,5.2692,77.00,22,18,2.30,NE,1615696038


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = perfect_temperature.loc[:,["City", "Country", "Lat", "Lng"]]
hotel_df["Hotel Name"] = ""

hotel_df

,City,Country,Lat,Lng,Hotel Name
118,Touros,BR,-5.1989,-35.4608,
150,Sokoni,TZ,-6.4333,39.5500,
171,East London,ZA,-33.0153,27.9116,
238,Maxixe,MZ,-23.8597,35.3472,
294,Pisco,PE,-13.7000,-76.2167,
459,Vila Velha,BR,-20.3297,-40.2925,
469,El Salvador,SV,13.8333,-88.9167,
493,Tahoua,NE,14.8888,5.2692,


In [6]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
#   pprint.pprint(response)
#   break
   
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("No hotel")
        
    print("------------")


Retrieving Results for Index 118: Touros.
Closest hotel in Touros is Hotel Vila Gale Touros.
------------
Retrieving Results for Index 150: Sokoni.
Closest hotel in Sokoni is Clove Island Zanzibar Villas & Spa.
------------
Retrieving Results for Index 171: East London.
Closest hotel in East London is Blue Lagoon Hotel.
------------
Retrieving Results for Index 238: Maxixe.
Closest hotel in Maxixe is Sentidos Beach Retreat.
------------
Retrieving Results for Index 294: Pisco.
Closest hotel in Pisco is PiscoMar Peru.
------------
Retrieving Results for Index 459: Vila Velha.
Closest hotel in Vila Velha is Golden Tulip Porto Vitória.
------------
Retrieving Results for Index 469: El Salvador.
No hotel
------------
Retrieving Results for Index 493: Tahoua.
Closest hotel in Tahoua is Hotel Etab.
------------


In [7]:
 hotel_df

,City,Country,Lat,Lng,Hotel Name
118,Touros,BR,-5.1989,-35.4608,Hotel Vila Gale Touros
150,Sokoni,TZ,-6.4333,39.5500,Clove Island Zanzibar Villas & Spa
171,East London,ZA,-33.0153,27.9116,Blue Lagoon Hotel
238,Maxixe,MZ,-23.8597,35.3472,Sentidos Beach Retreat
294,Pisco,PE,-13.7000,-76.2167,PiscoMar Peru
459,Vila Velha,BR,-20.3297,-40.2925,Golden Tulip Porto Vitória
469,El Salvador,SV,13.8333,-88.9167,
493,Tahoua,NE,14.8888,5.2692,Hotel Etab


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))